# Relatório de desenvolvimento do módulo `doc_data` versão `0.5.7`
**Autor**: Caio Geraldes

## Introdução

O módulo `doc_data`, escrito em `Python`, tem como função auxiliar a coleta dos dados necessários para a investigação proposta em meu doutorado.
O módulo se aproveita do corpus anotado `Diorisis` e do pacote de processamento de linguagem natural `stanza` para produzir árvores de dependência que filtram sentenças em que seria plausível encontrar atração de caso de predicados secundários de orações infinitivas do grego antigo.
Os detalhes de funcionamento do módulo está especificado no repositório no GitHub [caiogeraldes/doc_data](www.github.com/caiogeraldes/doc_data) e nos submódulos, funções e classes que compõem o projeto.

# Metodologia

O módulo `doc_data` segue, na versão `0.5.7`, as seguintes etapas para filtrar os dados:

1. Os textos disponibilizados no corpus `Diorisis` são separados em um ficheiro próprio;
2. Os textos são processados para retirar apenas os tokens de texto e metadata (autoria, documento etc);
3. Os tokens são reprocessados pelo modelo treinado com o corpus `Perseus` com ajuda do pacote `stanza`, produzindo sentenças e seus constituintes tokenizados com relações de dependência, análise morfológica e indexação;
4. Os tokens são armazenados em um banco de dados `NoSQL`, neste caso um `MongoDB`, para permitir filtragens mais precisas;
5. Filtram-se os tokens:
    - que pertençam a sentenças contendo um verbo principal de interesse (`mvi` = *main verb of interest*), selecionados em [mvi.csv](https://github.com/caiogeraldes/doc_data/blob/main/data/mvi.csv);
    - que pertençam a sentenças contendo um infinitivo dependente do `mvi`;
    - que pertençam a sentenças contendo um dativo ou genitivo dependente do `mvi`.
    
## Efeitos da filtragem

O código abaixo permite inspecionar o resultado da filtragem.

In [7]:
from doc_data.main import MONGO
from doc_data.db import mongo
from doc_data.tools import gen_sent, gen_meta, gen_hits
db = mongo(MONGO)
tokens = db.tokens
interest_tokens = db.interest_tokens
print(f"Sentenças totais:\t\t{len(tokens.distinct('text-sentence'))}")
print(f"Tokens totais:\t\t\t{tokens.count_documents({})}")
print(f"Sentenças após filtragem:\t{len(interest_tokens.distinct('text-sentence'))}")
print(f"Tokens após filtragem:\t\t{interest_tokens.count_documents({})}")

Connected successfully
Sentenças totais:		79778
Tokens totais:			2408680
Sentenças após filtragem:	2077
Tokens após filtragem:		109180


O corpus inicial `tokens` contém $92.551$ sentenças contendo $2.855.303$ tokens.
Após os três passos de filtragem, o corpus final `interest_tokens`, contém $3.621$ sentenças contendo $199.656$ tokens, $3.9\%$ do número de sentenças original ($7\%$ do número de tokens original).

## Qualidade da filtragem

**PÉSSIMA**

Seleciono aleatoriamente abaixo 36 sentenças ($\simeq 1\%$ do total) filtradas para averiguar se o método de filtragem produziu dados utilizáveis (a `seed` 36 foi escolhida arbitrariamente para que sempre se produzam as mesmas frases):

In [8]:
import random
random.seed(36)
xobj = db.xobjquery
tss = xobj.distinct("text-sentence")
for ts in random.choices(tss, k=36):
    print(gen_meta(interest_tokens, ts))
    print(gen_hits(interest_tokens, ts, [db.mviquery, db.infquery, xobj]))
    print(gen_sent(interest_tokens, ts))
    print()

Demosthenes (0014) - On the Crown (018)
{'mviquery': ['συμβέβηκε'], 'infquery': ['ἀπολωλεκέναι'], 'xobjquery': ['πλήθεσιν']}
εἶτʼ , οἶμαι , συμβέβηκε τοῖς μὲν πλήθεσιν ἀντὶ τῆς πολλῆς καὶ ἀκαίρου ῥᾳθυμίας τὴν ἐλευθερίαν ἀπολωλεκέναι , τοῖς δὲ προεστηκόσι καὶ τἄλλα πλὴν ἑαυτοὺς οἰομένοις πωλεῖν πρώτους ἑαυτοὺς πεπρακόσιν αἰσθέσθαι ·

Xenophon (0032) - Anabasis (006)
{'mviquery': ['ἔδοξε', 'δοῦναι'], 'infquery': ['δοῦναι'], 'xobjquery': ['στρατιώταις', 'αὐτῷ']}
ἐκ τούτου ἔδοξε τοῖς στρατιώταις δοῦναι αὐτῷ τὰ δημόσια πρόβατα ·

Plato (0059) - Protagoras (022)
{'mviquery': ['δοκεῖ', 'δοκεῖ'], 'infquery': ['πεποιῆσθαι', 'πεποιῆσθαι'], 'xobjquery': ['σοι', 'σοι']}
πότερον οὖν καλῶς σοι δοκεῖ πεποιῆσθαι καὶ ὀρθῶς , ἢ οὔ ; _ πάνυ , ἔφην ἐγώ , καλῶς τε καὶ ὀρθῶς . _ δοκεῖ δέ σοι καλῶς πεποιῆσθαι , εἰ ἐναντία λέγει αὐτὸς αὑτῷ ὁ ποιητής ;

Lysias (0540) - For Polystratus (020)
{'mviquery': ['δώσετε'], 'infquery': ['γενέσθαι'], 'xobjquery': ['ὑμῖν']}
καὶ εἰ μὲν ξένος τις ἐλθὼν ὑμᾶς ἢ χρήματα ᾔτει 

{'mviquery': ['δοκεῖς'], 'infquery': ['εἴσεσθαι'], 'xobjquery': ['μοι']}
ἐπειδὴ δὲ τῶν τοιούτων ἅδην εἴχομεν , αὖθις ἐγὼ αὐτοὺς ἀνηρώτων τὰ τῇδε , περὶ φιλοσοφίας ὅπως ἔχοι τὰ νῦν , περί τε τῶν νέων , εἴ τινες ἐν αὐτοῖς διαφέροντες ἢ σοφίᾳ ἢ κάλλει ἢ ἀμφοτέροις ἐγγεγονότες εἶεν . καὶ ὁ Κριτίας ἀποβλέψας πρὸς τὴν θύραν , ἰδών τινας νεανίσκους εἰσιόντας καὶ λοιδορουμένους ἀλλήλοις καὶ ἄλλον ὄχλον ὄπισθεν ἑπόμενον , περὶ μὲν τῶν καλῶν , ἔφη , ὦ Σώκρατες , αὐτίκα μοι δοκεῖς εἴσεσθαι ·

Plato (0059) - Theaetetus (006)
{'mviquery': ['ἐγχωρεῖ'], 'infquery': ['ἀλλοδοξεῖν'], 'xobjquery': ['δοξάζοντι']}
ἀναγκάζοιτο γὰρ ἂν ἐφάπτεσθαι καὶ οὗ μὴ δοξάζει . οὔτʼ ἄρʼ ἀμφότερα οὔτε τὸ ἕτερον δοξάζοντι ἐγχωρεῖ ἀλλοδοξεῖν . ὥστʼ εἴ τις ὁριεῖται δόξαν εἶναι ψευδῆ τὸ ἑτεροδοξεῖν , οὐδὲν ἂν λέγοι ·

Herodotus (0016) - Histories (001)
{'mviquery': ['ἐδόκεον'], 'infquery': ['κρατήσειν'], 'xobjquery': ['πολλῷ']}
κατὰ μέν νυν τὴν θάλασσαν ἑσσωμένοι ἦσαν τῷ θυμῷ , πεζῇ δὲ ἐδόκεον πολλῷ κρατήσειν τὸν Μαρδόνιον . 

In [15]:
import pandas as pd

In [36]:
df = pd.DataFrame.from_dict([{"metadata": gen_meta(interest_tokens, ts),
                              "sent": gen_sent(interest_tokens, ts),
                              "mvi": ";".join(gen_hits(interest_tokens, ts, [db.mviquery])["mviquery"]),
                              "inf": ";".join(gen_hits(interest_tokens, ts, [db.infquery])["infquery"]),
                              "xobj": ";".join(gen_hits(interest_tokens, ts, [db.xobjquery])["xobjquery"]),

                             } for ts in tss])

df.to_csv("dat.csv")

In [37]:
df

,metadata,sent,mvi,inf,xobj
0,Demosthenes (Apollodorus) (0014) - Against Ste...,"ὅταν γὰρ τῷ δούλῳ συνδικῇ τὸν ἀδελφὸν ἀτιμῶν ,...",προσῆκεν,θαυμάζεσθαι,αὐτῷ
1,Plato (0059) - Laws (034),αἱ πλαστικαὶ γὰρ καὶ ὅσαι πλεκτικαὶ τῶν τεχνῶν...,ἔδωκε,πορίζειν,ἀνθρώποις
2,Antiphon (0028) - Prosecution Of The Stepmothe...,πῶς οὖν ταύτην ἐλεεῖν ἄξιόν ἐστιν ἢ αἰδοῦς τυγ...,προσήκει,ἐλεεῖν,παθήμασι
3,Euripides (0006) - Iphigenia in Aulis (051),οὐκ ἄρα δοκεῖ σοι τάδε πονεῖν *̔ελλάδι ;,δοκεῖ,πονεῖν,*̔ελλάδι
4,Xenophon (0032) - Anabasis (006),Σοφαίνετον δὲ τὸν Στυμφάλιον καὶ Σωκράτην τὸν ...,ἐδόκει,πορεύεσθαι,αὐτῷ
5,Xenophon (0032) - Anabasis (006),ἐμοὶ μὲν οὖν δοκεῖ παραγγεῖλαι μὲν ἀριστοποιεῖ...,δοκεῖ;παραγγεῖλαι;δοκεῖ;δοκεῖ,παραγγεῖλαι;ἀριστοποιεῖσθαι;βουλεύεσθαι;ὑπερβά...,ἐμοὶ;στρατιώταις
6,Thucydides (0003) - History (001),οἱ γὰρ τότε τῶν Σαμίων ἐπαναστάντες τοῖς δυνατ...,διδόντες;σημαίνουσι;ἐδόκουν,ἐναντιοῦσθαι,ξυνεστῶσιν
7,Demosthenes (0014) - Against Leptines (020),τῆς δέ γʼ αἰσχύνης ὅλην ἀναπίμπλησι τὴν πόλιν ...,προσήκει,λελύσθαι,τοῖσδʼ
8,Xenophon (0032) - Hellenica (001),"καὶ ἐγὼ μέν , ἔφη , ἐπεὶ φίλος ἡμῖν γεγένησαι ...",συμβουλεύοιμʼ,ἄγεσθαι,σοι
